In [1]:
#TALLER

In [2]:
import pandas as pd
import numpy as np

In [3]:
#PUNTO 1

In [4]:
#Importando tambien los datos de la GEIH 2025, directamente desde el repositorio del profesor
df_cgsse = pd.read_csv("https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/cgsse.CSV"
                       ,encoding="latin", sep= ";") 
df_ft = pd.read_csv("https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Fuerza%20de%20trabajo.CSV"
                       ,encoding="latin", sep= ";", low_memory=False) 
df_ocupados = pd.read_csv("https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Ocupados.CSV"
                       ,encoding="latin", sep= ";", low_memory=False) 
df_desocupados = pd.read_csv("https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/No%20ocupados.CSV"
                       ,encoding="latin", sep= ";") 

In [5]:
# 1. Poblacion total de colombia: 
ptc = df_cgsse["FEX_C18"].sum()
print('Población total de colombia:', int(ptc))

Población total de colombia: 52045001


In [6]:
#Poblacion en edad de trabajar y en no edad de trabajar

sin_edad_trabajar = df_cgsse[df_cgsse['P6040']<15]['FEX_C18'].sum()
edad_trabajar = df_cgsse[df_cgsse['P6040']>=15]['FEX_C18']
total_edad_trabajar = df_cgsse[df_cgsse['P6040']>=15]['FEX_C18'].sum()

print('La poblacion total sin edad para trabajar es:',int(sin_edad_trabajar))
print('La poblacion total con edad para trabjar es:',int(total_edad_trabajar))


La poblacion total sin edad para trabajar es: 11391585
La poblacion total con edad para trabjar es: 40653416


In [7]:
#Calculado ahora la poblacion economicamente activa
pea = df_ft[df_ft['FT']==1]['FEX_C18'].sum()
print('La poblacion economicamente activa es:',int(pea))

La poblacion economicamente activa es: 25974846


In [8]:
#Calculo de Poblacion economicamente inactiva
pei = df_ft[df_ft['FFT']==1]['FEX_C18'].sum()
print('La poblacion economicamente inactiva es:',int(pei))


La poblacion economicamente inactiva es: 14678569


In [9]:
# Tasa de desempleo
desocupados = df_desocupados['FEX_C18'].sum()
td = (desocupados / pea) * 100
print('La tasa de desempleo es:', round(td, 2), '%')

La tasa de desempleo es: 65.06 %


In [10]:
#Cantiadad de personas desempleadas
D = df_desocupados[df_desocupados['DSI']==1]['FEX_C18'].sum()
print('La cantidad de personas desempleadas es:', int(D))

#Tasa de desempleados
td = (D / pea)
print(f'La tasa de desempleo es:{td*100} %')

La cantidad de personas desempleadas es: 2221893
La tasa de desempleo es:8.554020086155528 %


In [12]:
# Tasa de ocupados
O = df_ocupados['FEX_C18'].sum()
print('La cantidad de personas ocupadas es:', int(O))

#poblacion en edad de trabajar
pet = df_ft['FEX_C18'].sum()
print('La poblacion en edad de trabajar es:', int(pet))

#TASA DE OCUPADOS
to = (O / pet) 
print(f'La tasa de ocupados es: {to:.2%}')

La cantidad de personas ocupadas es: 23752952
La poblacion en edad de trabajar es: 40653416
La tasa de ocupados es: 58.43%


In [ ]:
#PUNTO 2

In [14]:
#Hacemos un merge para tener el sexo de los desocupados
df_desocupados_gen = pd.merge(
    df_desocupados[df_desocupados["DSI"]==1], 
    df_cgsse[['DIRECTORIO','SECUENCIA_P','ORDEN','P3271']], 
    on=['DIRECTORIO','SECUENCIA_P','ORDEN'], 
    how='left')
# Agrupamos por departamento y sexo
desocupados_sexo = (df_desocupados_gen.groupby(['DPTO','P3271'])['FEX_C18'].sum().reset_index().rename(columns={'FEX_C18':'desocupados'}))

# Ocupados también con sexo
df_ocupados_gen = pd.merge(
    df_ocupados, 
    df_cgsse[['DIRECTORIO','SECUENCIA_P','ORDEN','P3271']], 
    on=['DIRECTORIO','SECUENCIA_P','ORDEN'], 
    how='left')

ocupados_sexo = (df_ocupados_gen.groupby(['DPTO','P3271'])['FEX_C18'].sum().reset_index().rename(columns={'FEX_C18':'ocupados'}))

#Unimos ocupados y desocupados para calcular la PEA
pea_sexo = pd.merge(ocupados_sexo, desocupados_sexo, on=['DPTO','P3271'], how='outer').fillna(0)
pea_sexo['PEA'] = pea_sexo['ocupados'] + pea_sexo['desocupados']
#Tasa de desempleo (TD)
pea_sexo['TD'] = pea_sexo['desocupados'] / pea_sexo['PEA'] * 100

#Reemplazamos sexo por etiquetas
pea_sexo['Sexo'] = pea_sexo['P3271'].map({1:'Hombres', 2:'Mujeres'})
# Hacemos el print
print(pea_sexo[['DPTO','Sexo','TD']].head(20))

    DPTO     Sexo         TD
0      5  Hombres   4.886691
1      5  Mujeres   9.770415
2      8  Hombres   6.301949
3      8  Mujeres  10.232567
4     11  Hombres   9.821256
5     11  Mujeres   7.078481
6     13  Hombres   4.791098
7     13  Mujeres  12.028807
8     15  Hombres   6.967972
9     15  Mujeres   4.008697
10    17  Hombres   5.583183
11    17  Mujeres   7.677962
12    18  Hombres   2.676220
13    18  Mujeres  15.726909
14    19  Hombres   3.390661
15    19  Mujeres   5.146277
16    20  Hombres   5.991357
17    20  Mujeres  13.209651
18    23  Hombres   5.832048
19    23  Mujeres  19.837285


In [15]:
# Mayor y menor Tasa de Desempleo dada por sexo
td_max = pea_sexo.loc[pea_sexo.groupby("Sexo")["TD"].idxmax()]
td_min = pea_sexo.loc[pea_sexo.groupby("Sexo")["TD"].idxmin()]

print("Departamentos con MAYOR tasa de desempleo por sexo:")
print(td_max[['DPTO','Sexo','TD']])

print("\nDepartamentos con MENOR tasa de desempleo por sexo:")
print(td_min[['DPTO','Sexo','TD']])

Departamentos con MAYOR tasa de desempleo por sexo:
    DPTO     Sexo         TD
48    81  Hombres  18.057931
49    81  Mujeres  31.660892

Departamentos con MENOR tasa de desempleo por sexo:
    DPTO     Sexo        TD
56    91  Hombres  1.979606
57    91  Mujeres  1.874288


In [ ]:
#PUNTO 3

In [17]:
# Hacer merge para traer el género (P3271: 1=Hombres, 2=Mujeres) ---
df_desocupados_genero = pd.merge(df_desocupados_qrc,df_cgsse[["DIRECTORIO", "SECUENCIA_P", "ORDEN", "P3271"]], on=["DIRECTORIO", "SECUENCIA_P", "ORDEN"], how="left")

# Calcular hombres desocupados
PHDO = df_desocupados_genero[df_desocupados_genero["P3271"] == 1]["FEX_C18"].sum()
print(f"Hombres desempleados: {int(PHDO)}")

# Calcular mujeres desocupadas
PMDO = df_desocupados_genero[df_desocupados_genero["P3271"] == 2]["FEX_C18"].sum()
print(f"Mujeres desempleadas: {int(PMDO)}")

Hombres desempleados: 54096
Mujeres desempleadas: 49509


In [ ]:
#CLASIFICAMOS LOS GRUPOS DE EDAD
# Para cada uno de los departamentos, cual es el grupo de edad con mayor y menor tasa de desempleo? 

def clasificar_edad(x):
    if 18 <= x < 24:
        return "18-24"
    elif 24 <= x < 30:
        return "24-30"
    elif 30 <= x < 40:
        return "30-40"
    elif 40 <= x < 50:
        return "40-50"
    elif x >= 60:
        return "60+"
    else:
        return "Otros"  # Para menores de 18 y personas entre 50-59 que no clasifican

In [19]:
#Armamos la PEA por departamento y grupo de edad
#Hacemos merge para traer la edad (P6040) tanto en ocupados como en desocupados.
#Agrupamos por DPTO y grupo_edad.
#Calculamos TD como antes
# Crear columna grupo de edad en df_cgsse
df_cgsse["grupo_edad"] = df_cgsse["P6040"].apply(clasificar_edad)

# Desocupados con edad
df_desocupados_gen = pd.merge(df_desocupados[df_desocupados["DSI"]==1], df_cgsse[["DIRECTORIO","SECUENCIA_P","ORDEN","grupo_edad"]], on=["DIRECTORIO","SECUENCIA_P","ORDEN"], how="left")

desocupados_edad = (df_desocupados_gen.groupby(["DPTO","grupo_edad"])["FEX_C18"].sum().reset_index().rename(columns={"FEX_C18":"desocupados"}))

# Ocupados con edad
df_ocupados_gen = pd.merge(df_ocupados, df_cgsse[["DIRECTORIO","SECUENCIA_P","ORDEN","grupo_edad"]], on=["DIRECTORIO","SECUENCIA_P","ORDEN"], how="left")

ocupados_edad = (df_ocupados_gen.groupby(["DPTO","grupo_edad"])["FEX_C18"].sum().reset_index().rename(columns={"FEX_C18":"ocupados"}))

# PEA y TD
pea_edad = pd.merge(ocupados_edad, desocupados_edad, on=["DPTO","grupo_edad"], how="outer").fillna(0)
pea_edad["PEA"] = pea_edad["ocupados"] + pea_edad["desocupados"]
pea_edad["TD"] = pea_edad["desocupados"] / pea_edad["PEA"] * 100

In [28]:
# Filtramos solo los grupos válidos
pea_edad = pea_edad[pea_edad["grupo_edad"].isin(["18-24","24-30","30-40","40-50","60+"])] 

# Para cada departamento, el grupo de mayor y menor TD
mayores = pea_edad.loc[pea_edad.groupby("DPTO")["TD"].idxmax()]
menores = pea_edad.loc[pea_edad.groupby("DPTO")["TD"].idxmin()]

print("Grupo con MAYOR tasa de desempleo por departamento:")
print(mayores[["DPTO","grupo_edad","TD"]])


Grupo con MAYOR tasa de desempleo por departamento:
     DPTO grupo_edad         TD
0       5      18-24  17.685962
6       8      18-24  22.406918
12     11      18-24  17.797990
18     13      18-24  29.157386
25     15      24-30  14.491221
30     17      18-24   9.972584
36     18      18-24  21.686963
43     19      24-30   7.775308
48     20      18-24  19.042303
54     23      18-24  31.660884
60     25      18-24  15.618810
66     27      18-24  21.102675
72     41      18-24  14.602874
79     44      24-30  24.685637
84     47      18-24  20.675536
90     50      18-24  13.483416
96     52      18-24  16.529172
102    54      18-24  28.074930
109    63      24-30  14.237924
114    66      18-24  15.173556
120    68      18-24  23.222696
127    70      24-30  22.884184
132    73      18-24  19.348020
138    76      18-24  17.821703
144    81      18-24  61.095429
150    85      18-24  27.966955
156    86      18-24  44.878068
163    88      24-30  23.449477
171    91      40-50

In [27]:
print("\nGrupo con MENOR tasa de desempleo por departamento:")
print(menores[["DPTO","grupo_edad","TD"]])


Grupo con MENOR tasa de desempleo por departamento:
     DPTO grupo_edad         TD
4       5        60+   1.199292
9       8      40-50   4.082692
16     11        60+   5.426223
22     13        60+   2.222537
28     15        60+   0.440514
34     17        60+   4.060989
40     18        60+   1.244864
46     19        60+   0.539495
52     20        60+   0.644501
58     23        60+   4.322568
64     25        60+   4.202708
70     27        60+   6.162454
76     41        60+   0.950585
82     44        60+   0.466427
88     47        60+   3.495022
94     50        60+   3.972646
100    52        60+   2.122516
106    54        60+   4.730661
111    63      40-50   6.506359
118    66        60+   3.639861
124    68        60+   4.564159
130    70        60+   2.869353
136    73        60+   9.458632
142    76        60+   5.954849
147    81      40-50  12.725505
154    85        60+   0.000000
158    86      30-40  10.837416
165    88      40-50   6.261247
168    91      18-2

In [ ]:
#En la mayoría de departamentos, los jóvenes de entre 18 y 24 años son el grupo con la tasa más alta de desempleo, lo cual evidencia los obstáculos que enfrenta esta población para conseguir trabajo.  En ciertas regiones, como Bolívar, Valle del Cauca y Norte de Santander, la franja de 24 a 30 años es la que más se ve afectada, lo cual demuestra que la vulnerabilidad también afecta a los adultos jóvenes.

In [ ]:
#PUNTO 4

In [32]:
#Filtrar mujeres de 18 a 24 años de edad
df_cgsse["grupo_edad"] = df_cgsse["P6040"].apply(clasificar_edad)
df_mujeres_oc = df_ocupados_gen[(df_ocupados_gen["P3271"]==2) & (df_ocupados_gen["grupo_edad"]=="18-24")]
df_mujeres_des = df_desocupados_gen[(df_desocupados_gen["P3271"]==2) & (df_desocupados_gen["grupo_edad"]=="18-24")]

In [31]:
#Merge de ocupados y desocupados con sexo y edad
df_ocupados_gen = pd.merge(df_ocupados, df_cgsse[["DIRECTORIO","SECUENCIA_P","ORDEN","P3271","grupo_edad"]], on=["DIRECTORIO","SECUENCIA_P","ORDEN"], how="left")
df_desocupados_gen = pd.merge(df_desocupados[df_desocupados["DSI"]==1], df_cgsse[["DIRECTORIO","SECUENCIA_P","ORDEN","P3271","grupo_edad"]], on=["DIRECTORIO","SECUENCIA_P","ORDEN"], how="left")

In [33]:
#Agrupar por departamento
ocupadas = df_mujeres_oc.groupby("DPTO")["FEX_C18"].sum().reset_index().rename(columns={"FEX_C18":"ocupadas"})
desocupadas = df_mujeres_des.groupby("DPTO")["FEX_C18"].sum().reset_index().rename(columns={"FEX_C18":"desocupadas"})

In [36]:
#Calcular PEA y TD
pea = pd.merge(ocupadas, desocupadas, on="DPTO", how="outer").fillna(0)
pea["PEA"] = pea["ocupadas"] + pea["desocupadas"]
pea["TD"] = pea["desocupadas"] / pea["PEA"] * 100

In [37]:
#Ordenar y mostrar los 3 departamentos con mayor TD
td_mujeres_18_24_min = pea.sort_values("TD").head(3)
print(td_mujeres_18_24_min)

    DPTO     ocupadas  desocupadas          PEA        TD
31    97   111.385759     0.000000   111.385759  0.000000
28    91   251.281733     0.000000   251.281733  0.000000
18    63  7322.443307   310.224229  7632.667536  4.064427


In [ ]:
#Vaupés (0%), Amazonas (0%) y Quindío (4.06%) son los tres departamentos con la tasa de desempleo más baja en mujeres de 18 a 24 años, lo que demuestra que las mujeres jóvenes en estas áreas tienen menos obstáculos para acceder al mercado laboral.

In [ ]:
#PUNTO 5 

In [41]:
# Filtrar ocupados en los departamentos a analizar
deptos_interes = [66, 17, 63, 5, 76]  # 66=Risaralda, 17=Caldas, 63=Quindío, 5=Antioquia, 76=Valle del Cauca
df_ingresos = df_ocupados[df_ocupados["DPTO"].isin(deptos_interes)]

# Calcular el promedio ponderado de ingresos laborales (INGLABO)
promedios_ingresos = (df_ingresos.groupby("DPTO").apply(lambda x: (x["INGLABO"] * x["FEX_C18"]).sum() / x["FEX_C18"].sum()).reset_index(name="Ingreso_promedio"))

# Reemplazar códigos de departamentos por nombres
mapa_dptos = {66:"Risaralda", 17:"Caldas", 63:"Quindío", 5:"Antioquia", 76:"Valle del Cauca"}
promedios_ingresos["Departamento"] = promedios_ingresos["DPTO"].map(mapa_dptos)

# Reordenar columnas
promedios_ingresos = promedios_ingresos[["Departamento","Ingreso_promedio"]]

print(promedios_ingresos)

      Departamento  Ingreso_promedio
0        Antioquia      2.090819e+06
1           Caldas      1.958683e+06
2          Quindío      1.637336e+06
3        Risaralda      1.621205e+06
4  Valle del Cauca      1.698392e+06


/tmp/ipykernel_2089/351722446.py:6: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  promedios_ingresos = (df_ingresos.groupby("DPTO").apply(lambda x: (x["INGLABO"] * x["FEX_C18"]).sum() / x["FEX_C18"].sum()).reset_index(name="Ingreso_promedio"))


In [ ]:
#Los hallazgos revelan que el ingreso laboral promedio más elevado de los departamentos estudiados en 2025 es el de Antioquia (≈2,09 millones de pesos), y luego está Caldas (≈1,96 millones).  Por otro lado, los ingresos más bajos se registran en Quindío (≈1,63 millones) y Risaralda (≈1,62 millones).  Esto muestra una disparidad regional en el mercado de trabajo, donde Antioquia tiene la mayor retribución media, a diferencia de los departamentos del Eje Cafetero, que tienen ingresos más modestos.

In [ ]:
#PUNTO 6 

In [42]:
# Definir función para agrupar edades
def clasificar_edad(x):
    if 18 <= x < 24:
        return "18-24"
    elif 24 <= x < 30:
        return "24-30"
    elif 30 <= x < 40:
        return "30-40"
    elif 40 <= x < 50:
        return "40-50"
    elif x >= 60:
        return "60+"
    else:
        return None

# Crear variable de grupo de edad
df_cgsse["grupo_edad"] = df_cgsse["P6040"].apply(clasificar_edad)

In [43]:
# Merge con ocupados para traer sexo y edad
df_ocupados_gen = pd.merge(df_ocupados, df_cgsse[["DIRECTORIO","SECUENCIA_P","ORDEN","P3271","grupo_edad"]], on=["DIRECTORIO","SECUENCIA_P","ORDEN"],how="left")

# Filtrar departamentos de interés
deptos_interes = [66, 17, 63, 5, 76]
df_filtrado = df_ocupados_gen[df_ocupados_gen["DPTO"].isin(deptos_interes)]

In [44]:
# Calcular promedio ponderado de ingresos por depto, sexo y grupo de edad
prom_ingresos = (df_filtrado.groupby(["DPTO","P3271","grupo_edad"]).apply(lambda x: (x["INGLABO"] * x["FEX_C18"]).sum() / x["FEX_C18"].sum()).reset_index(name="Ingreso_promedio"))

/tmp/ipykernel_2089/4095260200.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  prom_ingresos = (df_filtrado.groupby(["DPTO","P3271","grupo_edad"]).apply(lambda x: (x["INGLABO"] * x["FEX_C18"]).sum() / x["FEX_C18"].sum()).reset_index(name="Ingreso_promedio"))


In [45]:
# Mapear nombres de departamentos y sexo
mapa_dptos = {66:"Risaralda", 17:"Caldas", 63:"Quindío", 5:"Antioquia", 76:"Valle del Cauca"}
prom_ingresos["Departamento"] = prom_ingresos["DPTO"].map(mapa_dptos)
prom_ingresos["Sexo"] = prom_ingresos["P3271"].map({1:"Hombres", 2:"Mujeres"})

In [46]:
prom_ingresos = prom_ingresos[["Departamento","Sexo","grupo_edad","Ingreso_promedio"]]
print(prom_ingresos.head(20))

   Departamento     Sexo grupo_edad  Ingreso_promedio
0     Antioquia  Hombres      18-24      1.364005e+06
1     Antioquia  Hombres      24-30      1.828144e+06
2     Antioquia  Hombres      30-40      2.159206e+06
3     Antioquia  Hombres      40-50      2.771747e+06
4     Antioquia  Hombres        60+      1.738620e+06
5     Antioquia  Mujeres      18-24      1.418639e+06
6     Antioquia  Mujeres      24-30      1.837916e+06
7     Antioquia  Mujeres      30-40      2.291801e+06
8     Antioquia  Mujeres      40-50      2.111094e+06
9     Antioquia  Mujeres        60+      1.510449e+06
10       Caldas  Hombres      18-24      2.216063e+06
11       Caldas  Hombres      24-30      1.754677e+06
12       Caldas  Hombres      30-40      1.762301e+06
13       Caldas  Hombres      40-50      2.470039e+06
14       Caldas  Hombres        60+      2.309864e+06
15       Caldas  Mujeres      18-24      1.133456e+06
16       Caldas  Mujeres      24-30      1.832716e+06
17       Caldas  Mujeres    

In [ ]:
#En la mayoría de las situaciones, los ingresos laborales se incrementan con la edad. Tanto hombres como mujeres llegan a su máximo entre los 40 y los 50 años.  Además, se ha visto que, a excepción de algunas situaciones específicas como la de los hombres jóvenes (18-24 años) en Caldas, los hombres tienden a tener un ingreso medio más alto que las mujeres en los mismos grupos etarios. Esto muestra una diferencia salarial entre géneros en las regiones estudiadas.

In [ ]:
# PUNTO 7

In [47]:
print(df_ocupados.columns)

# Calcular promedio ponderado de ingresos laborales por oficio (OFICIO_C8)
ingresos_ciuo = (df_ocupados.groupby("OFICIO_C8").apply(lambda x: (x["INGLABO"] * x["FEX_C18"]).sum() / x["FEX_C18"].sum()).reset_index(name="Ingreso_promedio"))

# Ordenar de mayor a menor ingreso
ingresos_ciuo = ingresos_ciuo.sort_values("Ingreso_promedio", ascending=False)

# Mostrar los 10 oficios mejor pagados
print(ingresos_ciuo.head(10))

Index(['PERIODO', 'MES', 'PER', 'DIRECTORIO', 'SECUENCIA_P', 'ORDEN', 'HOGAR',
       'REGIS', 'AREA', 'CLASE',
       ...
       'P1881', 'P1881S1', 'P1882', 'P7240', 'P7240S1', 'OCI', 'INGLABO',
       'RAMA2D_R4', 'RAMA4D_R4', 'OFICIO_C8'],
      dtype='object', length=202)
     OFICIO_C8  Ingreso_promedio
63        2267      5.176025e+07
25        1349      1.673692e+07
93        2523      1.100000e+07
96        2612      1.040657e+07
94        2529      1.015048e+07
156       3321      9.351115e+06
52        2212      8.946374e+06
7         1211      8.923134e+06
90        2513      8.852754e+06
12        1222      8.541606e+06


/tmp/ipykernel_2089/1933472876.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ingresos_ciuo = (df_ocupados.groupby("OFICIO_C8").apply(lambda x: (x["INGLABO"] * x["FEX_C18"]).sum() / x["FEX_C18"].sum()).reset_index(name="Ingreso_promedio"))


In [ ]:
#PUNTO 8

In [48]:
# Oficio elegido (2212 = Médicos especialistas)
codigo_oficio = 2212

# Departamentos de interés
deptos_interes = [17, 63, 5, 76]

# Merge para traer sexo (P3271) desde df_cgsse
df_ocupados_gen = pd.merge(df_ocupados, df_cgsse[["DIRECTORIO","SECUENCIA_P","ORDEN","P3271"]], on=["DIRECTORIO","SECUENCIA_P","ORDEN"], how="left")

# Filtrar por oficio y departamentos
df_filtrado = df_ocupados_gen[(df_ocupados_gen["OFICIO_C8"] == codigo_oficio) & (df_ocupados_gen["DPTO"].isin(deptos_interes))]

# Calcular promedio ponderado de ingresos por sexo y departamento
promedios = (df_filtrado.groupby(["DPTO","P3271"]).apply(lambda x: (x["INGLABO"] * x["FEX_C18"]).sum() / x["FEX_C18"].sum()).reset_index(name="Ingreso_promedio"))

# Mapear nombres
mapa_dptos = {17:"Caldas", 63:"Quindío", 5:"Antioquia", 76:"Valle del Cauca"}
promedios["Departamento"] = promedios["DPTO"].map(mapa_dptos)
promedios["Sexo"] = promedios["P3271"].map({1:"Hombres", 2:"Mujeres"})

# Reordenar columnas
promedios = promedios[["Departamento","Sexo","Ingreso_promedio"]]

print(promedios)

      Departamento     Sexo  Ingreso_promedio
0        Antioquia  Hombres      1.568818e+07
1           Caldas  Hombres      4.913083e+06
2           Caldas  Mujeres      1.000000e+07
3          Quindío  Hombres      4.693600e+06
4          Quindío  Mujeres      6.563233e+06
5  Valle del Cauca  Hombres      7.000000e+06
6  Valle del Cauca  Mujeres      1.291129e+07


/tmp/ipykernel_2089/1318668966.py:14: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  promedios = (df_filtrado.groupby(["DPTO","P3271"]).apply(lambda x: (x["INGLABO"] * x["FEX_C18"]).sum() / x["FEX_C18"].sum()).reset_index(name="Ingreso_promedio"))


Los hallazgos evidencian disparidades significativas entre hombres y mujeres en la ocupación que se estudia.  En Antioquia, los hombres tienen ingresos mucho más altos (aproximadamente 15,7 millones).  En contraste, en Valle del Cauca, Caldas y Quindío las mujeres ganan más que los hombres; de hecho, en algunas ocasiones sus ingresos llegan a ser el doble (por ejemplo, en Valle del Cauca: 12.9 millones contra 7 millones).

 Para concluir, sí hay diferencias importantes entre géneros, aunque no son uniformes: en algunos departamentos predominan los hombres y en otros las mujeres, lo que demuestra una variación de la brecha salarial según el territorio dentro del mismo oficio.

In [ ]:
# PUNTO 9

In [49]:
# Filtrar ocupados del Quindío y unir sexo
df_ocupados_gen = pd.merge(df_ocupados, df_cgsse[["DIRECTORIO","SECUENCIA_P","ORDEN","P3271"]], on=["DIRECTORIO","SECUENCIA_P","ORDEN"], how="left")

df_quindio = df_ocupados_gen[df_ocupados_gen["DPTO"] == 63]

# Calcular promedio ponderado por oficio y sexo
prom_quindio = (df_quindio.groupby(["OFICIO_C8","P3271"]).apply(lambda x: (x["INGLABO"] * x["FEX_C18"]).sum() / x["FEX_C18"].sum()).reset_index(name="Ingreso_promedio"))

# Mapear sexo
prom_quindio["Sexo"] = prom_quindio["P3271"].map({1:"Hombres", 2:"Mujeres"})

# Oficio con mayor y menor ingreso para hombres
max_hombres = prom_quindio[prom_quindio["Sexo"]=="Hombres"].sort_values("Ingreso_promedio", ascending=False).head(1)
min_hombres = prom_quindio[prom_quindio["Sexo"]=="Hombres"].sort_values("Ingreso_promedio", ascending=True).head(1)

# Oficio con mayor y menor ingreso para mujeres
max_mujeres = prom_quindio[prom_quindio["Sexo"]=="Mujeres"].sort_values("Ingreso_promedio", ascending=False).head(1)
min_mujeres = prom_quindio[prom_quindio["Sexo"]=="Mujeres"].sort_values("Ingreso_promedio", ascending=True).head(1)

print("Mayor salario Hombres:\n", max_hombres)
print("\nMenor salario Hombres:\n", min_hombres)
print("\nMayor salario Mujeres:\n", max_mujeres)
print("\nMenor salario Mujeres:\n", min_mujeres)

Mayor salario Hombres:
     OFICIO_C8  P3271  Ingreso_promedio     Sexo
24       2142      1      7.986427e+06  Hombres

Menor salario Hombres:
      OFICIO_C8  P3271  Ingreso_promedio     Sexo
183       7111      1               0.0  Hombres

Mayor salario Mujeres:
     OFICIO_C8  P3271  Ingreso_promedio     Sexo
35       2212      2      6.563233e+06  Mujeres

Menor salario Mujeres:
      OFICIO_C8  P3271  Ingreso_promedio     Sexo
206       7513      2               0.0  Mujeres


/tmp/ipykernel_2089/4032156403.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  prom_quindio = (df_quindio.groupby(["OFICIO_C8","P3271"]).apply(lambda x: (x["INGLABO"] * x["FEX_C18"]).sum() / x["FEX_C18"].sum()).reset_index(name="Ingreso_promedio"))


2142 - Ingenieros electrónicos es la ocupación masculina mejor pagada en el Quindío, con un salario promedio de 7,98 millones de pesos.

 7111 - Albañiles es la ocupación con el salario más bajo de los hombres, mostrando cero ingresos. Esto probablemente se deba a que en la muestra no hay registros válidos o a que los ingresos han sido subdeclarados.

 2212 – Médicos especialistas es la ocupación con el ingreso más alto entre las mujeres, con una media de 6,56 millones de pesos.
 La categoría con menos ingresos en las mujeres es la de "Trabajadoras de preparación de alimentos callejeros", que se presenta con un valor cero, lo cual también indica que no se reportaron ingresos en la encuesta.